In [47]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_rows', 500)

In [54]:
import pandas as pd

df1 = pd.read_csv("./data/0401/nbroad-v2.csv")
df2 = pd.read_csv("./data/0401/gemma_suppl_rewrite.csv")

df2['rewrite_prompt'] = df2['rewrite_prompts']

df = pd.concat([df1[['rewrite_prompt']], df2[['rewrite_prompt']]])

# df = pd.read_csv("./data/0304/seed_final.csv")

df = df[df['rewrite_prompt'].str.split(" ").apply(len) <= 15]
df = df[~df['rewrite_prompt'].str.contains(r"if|preserving|maintaining|/|\(|\.|\n", regex=True)]
df = df[~df['rewrite_prompt'].str.endswith("-")]
df = df[~df['rewrite_prompt'].str.contains(f"one act|Christianity|back-alley city|first line as|no exceptions|verbs except|with a twist|every moment is|story titled|watched three episodes|being replaced with|make it clear that|character-customization-menu|or another|completely unknown to|but|assuming|various popular franchises", regex=True)]
df['rewrite_prompt'] = df['rewrite_prompt'].str.replace(r"\.$", "", regex=True)
df['rewrite_prompt'] = df['rewrite_prompt'].apply(lambda x: x.strip())

print(df.shape)
prompts = df['rewrite_prompt'].tolist()

df.iloc[:12]['rewrite_prompt']

(1103, 1)


1                                                       Rewrite the essay as a dramatic play
2                Rewrite the story with all the themes and settings being Star Wars inspired
7                                 Rewrite the essay, making it a sappy Hallmark movie script
8                          Rewrite the prompt as a fantasy version of the Great Race Of Yith
12                                         Rewrite the story with extreme sarcasm throughout
15    Rewrite the story combining elements of a Disney Princess story with "Game of Thrones"
16                                             Rewrite the story as a cheery children's book
18                                                             Rewrite the story as a comedy
19                   Rewrite this as a science fiction story with machines instead of humans
21                            Rewrite the essay as a 50s style crime noir for your next film
24                                                     Rewrite the sto

In [ ]:
# encode
from sentence_transformers import SentenceTransformer

t5_base = SentenceTransformer('sentence-t5-base')

embeddings = t5_base.encode(df['rewrite_prompt'].tolist()).tolist()

embeddings

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

def diverse_subset(vectors, k):
    # Convert vectors to numpy array
    X = np.array(vectors)
    
    # Perform K-means clustering
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(X)
    
    # Get the cluster labels
    cluster_labels = kmeans.labels_
    
    # Find the centroids of each cluster
    centroids = kmeans.cluster_centers_
    
    # Initialize a list to store the diverse subset
    diverse_subset = []
    
    # Select one vector from each cluster to include in the diverse subset
    for i in range(k):
        cluster_indices = np.where(cluster_labels == i)[0]
        if len(cluster_indices) > 0:
            # Choose a representative vector from the cluster (e.g., centroid)
            representative_index = np.argmin(np.linalg.norm(X[cluster_indices] - centroids[i], axis=1))
            diverse_subset.append(cluster_indices[representative_index])
    
    return diverse_subset

subset = diverse_subset(embeddings, 4)
print("Diverse subset:", len(subset))

In [ ]:
df_shuffle = df.iloc[subset].sample(frac=1).reset_index(drop=True)
df_shuffle.to_csv("./data/0401/nbroad-v2-gemma_suppl_rewrite.csv", index=False, columns=['rewrite_prompt'])

df_shuffle.iloc[0]['rewrite_prompt']

# Build predict datasets

In [55]:
import random

random.seed(42)

# df_prompts = pd.read_csv("./data/0401/nbroad-v2-gemma_suppl_rewrite.csv")
df_text = pd.read_csv("./data/0401/150_suppl_original_text.csv")
df_text = df_text[df_text['original_text'].str.split(" ").apply(len) <= 100]

# prompts = df_prompts.iloc[:200]['rewrite_prompt'].tolist()

df_rows = []
for p in prompts:
    texts = df_text.iloc[random.choices(range(len(df_text)), k=1)]['original_text'].to_list()
    t = texts[0]
    df_rows.append({'rewrite_prompt': p, 'original_text': t})

df = pd.DataFrame(df_rows)
df.to_csv("./data/0407/validate_dataset.csv", index=False)